In [ ]:
# script to collect all of the revised kinetic calculations for existing Disproportionation training reactions:
# and put them together into a database


In [1]:
import os
import re
import glob

import rmgpy.chemkin
import rmgpy.data.kinetics

import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict


import rmgpy.data.thermo
import rmgpy.data.kinetics
import rmgpy.reaction
import rmgpy.species
import rmgpy

import sys
sys.path.append('/work/westgroup/harris.se/autoscience/autoscience_workflow/workflow/scripts/kinetics')
import kineticfun

In [2]:
DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft/"
kinetics_libs = glob.glob(os.path.join(DFT_DIR, 'kinetics', 'reaction*', 'arkane', 'RMG_libraries'))

In [3]:
# only include reactions that belong to the disproportionation family

# Load the base model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model'
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(species_list)} species, {len(reaction_list)} reactions')



823758660.py:8 <module> INFO Thermo file has default temperature range 300.0 to 1000.0 and 1000.0 to 5000.0


110 species, 1822 reactions


In [4]:
# Load the Arkane reactions
entries = []
for i, lib_path in enumerate(kinetics_libs):
    matches = re.search('reaction_([0-9]{4})', lib_path)
    reaction_index = int(matches[1])
    ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
    ark_kinetics_database.load_libraries(lib_path)
    
    # TODO fix bug related to load_libraries not getting the actual name
    for key in ark_kinetics_database.libraries[''].entries.keys():
        entry = ark_kinetics_database.libraries[''].entries[key]
        entry.index = reaction_index
        entries.append(entry)
print(len(entries), 'entries found')

database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_0018/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_0127/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_0180/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_0211/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_0213/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library

database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_0748/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_0749/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_0754/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_0784/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_0793/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library

database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_1410/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_1509/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_1529/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_1625/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library  from /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_1638/arkane/RMG_libraries/reactions.py...
database.py:255 load_libraries INFO Loading kinetics library

105 entries found


In [5]:
# load the disproportionation family
# Load Kinetics Database (quick, automatically loads recommended families)
family = 'Disproportionation'
ref_library_path = os.path.join(rmgpy.settings['database.directory'], 'kinetics')
ref_database = rmgpy.data.kinetics.KineticsDatabase()
ref_database.load(
    ref_library_path,
    libraries=[],
    families=[family]
)
training_depo = ref_database.families['Disproportionation'].get_training_depository()
print(len(training_depo.entries), 'training reactions')


137 training reactions


In [6]:
for key in training_depo.entries: 
    for revised_entry in entries:
        if revised_entry.item.is_isomorphic(training_depo.entries[key].item):
            rmg_reaction_number = kineticfun.reaction_smiles2index(kineticfun.reaction2smiles(revised_entry.item))
            print(f'changing kinetics for reaction {rmg_reaction_number}\t{revised_entry}')
            training_depo.entries[key].data = revised_entry.data
            

changing kinetics for reaction 563	C2H5(33) + C3H5-A(94) <=> C3H6(12) + C2H4(11)
changing kinetics for reaction 507	C2H5(33) + NC3H7(92) <=> C2H6(32) + C3H6(12)
changing kinetics for reaction 575	C3H5-A(94) + NC3H7(92) <=> C3H6(12) + C3H6(12)
changing kinetics for reaction 391	NC3H7(92) + C2H3(22) <=> C2H4(11) + C3H6(12)
changing kinetics for reaction 403	NC3H7(92) + OH(15) <=> H2O(8) + C3H6(12)
changing kinetics for reaction 784	C4H7(190) + O2(2) <=> HO2(16) + C4H6(194)
changing kinetics for reaction 1001	C4H7(191) + O2(2) <=> HO2(16) + C4H6(194)
changing kinetics for reaction 1001	C4H7(191) + O2(2) <=> HO2(16) + C4H6(194)


In [7]:
# Save the results over the previous reaction.py
training_dir = os.path.join(rmgpy.settings['database.directory'], 'kinetics', 'families', family, 'training')
training_depo.save(os.path.join(training_dir, 'reactions.py'))